In [1]:
import json
import os
import warnings

In [2]:
source_notebook_directory = './human-eval-bia/' # must end with /
# Specify the filename to save the .jsonl file
target_jsonl_filename = '../data/human-eval-bia.jsonl'

In [8]:
list_of_cases = []


# List all files in the current directory
files = os.listdir(source_notebook_directory)

# Iterate through the files and print names ending with .ipynb
for file in files:
    if file.endswith('.ipynb'):
        notebook_filename = source_notebook_directory + file
        
        # Load and parse the notebook
        with open(notebook_filename, 'r') as file:
            notebook = json.load(file)
        
        task_id = notebook_filename
        prompt = None
        canonical_solution = None
        entry_point = None
        test = None
        
        # Iterate through the cells and print the source of code cells
        for cell in notebook['cells']:
            if cell['cell_type'] == 'code':
                # Joining the lines of code for better readability
                code = ''.join(cell['source'])
                # print('\n\nCODE\n\n',code)
        
                if code.startswith('check('):
                    entry_point = code.strip().replace("check(","").replace(")","").strip()
                elif '"""' in code:
                    temp = code.split('"""')
                    canonical_solution = temp[-1]
                    temp[-1] = ""
                    prompt = '"""'.join(temp)
                elif 'def check(' in code:
                    test = code 
                elif len(code.strip()) == 0:
                    pass
                else:
                    sample = code[:20]
                    warnings.warn(f"I had issues reading a cell in {task_id} starting with ")
                    
        if prompt is None:
            warnings.warn(f"Couldn't extract prompt from {task_id}.")
        elif canonical_solution is None:
            warnings.warn(f"Couldn't extract canonical_solution from {task_id}.")
        elif entry_point is None:
            warnings.warn(f"Couldn't extract entry_point from {task_id}.")
        
        test_case = {
            'task_id':task_id,
            'prompt':prompt,
            'canonical_solution':canonical_solution,
            'entry_point':entry_point,
            'test':test
        }

        print(test_case)
        list_of_cases.append(test_case)

{'task_id': './human-eval-bia/human_eval_bia_0.ipynb', 'prompt': 'import skimage\nimport numpy as np\n\ndef label_binary_image_and_count_labels(binary_image):\n    """\n    Consumes as input a binary image, applies connected component labeling to it, \n    counts the labeled objects and returns their count as single number.\n    """', 'canonical_solution': '\n    label_image = skimage.measure.label(binary_image)\n\n    return len(np.unique(label_image)) - 1', 'entry_point': 'label_binary_image_and_count_labels', 'test': 'def check(candidate):\n    assert candidate(np.asarray([\n        [0,0,0,0,0],\n        [0,1,0,0,0],\n        [0,0,0,0,0],\n        [1,0,0,0,0],\n        [0,0,0,1,0],\n    ])) == 3\n\n    assert candidate(np.asarray([\n        [0,0,0,0,0],\n        [0,1,0,0,0],\n        [0,0,0,0,0],\n        [0,0,0,0,0],\n        [0,0,0,0,0],\n    ])) == 1\n\n    assert candidate(np.asarray([\n        [0,0,0,0,0],\n        [0,0,0,0,0],\n        [0,0,0,0,0],\n        [0,0,0,0,0],\n     

In [9]:
# Open the file in write mode and save the dictionaries
with open(target_jsonl_filename, 'w') as file:
    for dictionary in list_of_cases:
        # Convert dictionary to a JSON formatted string and write it
        json_str = json.dumps(dictionary)
        file.write(json_str + '\n')

print(f'Data successfully saved to {target_jsonl_filename}.')

Data successfully saved to ../data/human-eval-bia.jsonl.
